<a href="https://colab.research.google.com/github/fds-imane/Data_Marketing/blob/main/data_marketing_part3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 **Modélisation CLV Lumina & Co**

**Partie 1 : Elyna**

**Chargement de la data**

In [1]:
import pandas as pd

# Charger les fichiers CSV
customers = pd.read_csv('customers.csv')
transactions = pd.read_csv('transactions.csv')
campaigns = pd.read_csv('campaigns.csv')

print("Les 3 fichiers sont chargés avec succès.")

Les 3 fichiers sont chargés avec succès.


In [7]:
#vérifier la colonne invoice_date
transactions["invoice_date"] = pd.to_datetime(transactions["invoice_date"])
#retourner la première date du dataset:
start_date = transactions["invoice_date"].min()
print(start_date)
#Calculer automatiquement les périodes
obs_end_date = start_date + pd.DateOffset(months=12)
target_end_date = obs_end_date + pd.DateOffset(months=12)

print("features:", start_date, "->", obs_end_date)
print("Target:", obs_end_date, "->", target_end_date)

2007-07-06 12:20:00
features: 2007-07-06 12:20:00 -> 2008-07-06 12:20:00
Target: 2008-07-06 12:20:00 -> 2009-07-06 12:20:00


In [11]:
#Créer les dataset feature et target
#DataSet Feature
obs_data = transactions[
    (transactions["invoice_date"] >= start_date) &
    (transactions["invoice_date"] < obs_end_date)
]
#DataSet Target
target_data = transactions[
    (transactions["invoice_date"] >= obs_end_date) &
    (transactions["invoice_date"] < target_end_date)
]
#Vérification:
print(obs_data["invoice_date"].min(), obs_data["invoice_date"].max())
print(target_data["invoice_date"].min(), target_data["invoice_date"].max())

print(transactions["invoice_date"].min())
print(transactions["invoice_date"].max())

2007-07-06 12:20:00 2008-07-04 12:20:00
2008-07-06 12:20:00 2009-07-05 12:20:00
2007-07-06 12:20:00
2011-12-09 12:50:00


Créer la colonne amount (montant dépensé)

In [29]:
transactions["amount"] = transactions["unit_price"] * transactions["quantity"]

# aussi pour target_data
target_data = target_data.copy()
target_data["amount"] = target_data["unit_price"] * target_data["quantity"]



Calculer le clv futur

In [30]:
clv_future = (
    target_data
    .groupby("customer_id")["amount"]
    .sum()
)

In [31]:
# récupérer tous les clients de la période d’observation
all_clients = obs_data["customer_id"].dropna().unique()
all_clients_df = pd.DataFrame(all_clients, columns=["customer_id"])

# merge avec la CLV calculée
clv_future = all_clients_df.merge(
    clv_future,
    on="customer_id",
    how="left"
)

# remplacer les NaN par 0
clv_future["amount"] = clv_future["amount"].fillna(0)

# renommer en CLV_12M
clv_future.rename(columns={"amount": "CLV_12M"}, inplace=True)

#Vérification finale
print(clv_future.head())
print(clv_future.isna().sum())

   customer_id  CLV_12M
0      58969.0      0.0
1      25026.0      0.0
2      58917.0      0.0
3      58923.0      0.0
4      43966.0      0.0
customer_id    0
CLV_12M        0
dtype: int64


Partie 2: Feature engineering


Features RFM de base

Recency (jours depuis dernier achat)

In [32]:
snapshot_date = obs_data["invoice_date"].max()

recency = (
    obs_data.groupby("customer_id")["invoice_date"]
    .max()
    .reset_index()
)

recency["recency_days"] = (snapshot_date - recency["invoice_date"]).dt.days
recency = recency[["customer_id", "recency_days"]]

Frequency (nombre d’achats)

In [33]:
frequency = (
    obs_data.groupby("customer_id")
    .size()
    .reset_index(name="frequency")
)

Monetary (montant total dépensé)

In [34]:
obs_data["amount"] = obs_data["unit_price"] * obs_data["quantity"]

monetary = (
    obs_data.groupby("customer_id")["amount"]
    .sum()
    .reset_index(name="monetary")
)

/tmp/ipython-input-175/3491314317.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  obs_data["amount"] = obs_data["unit_price"] * obs_data["quantity"]


Panier moyen

In [35]:
avg_basket = monetary.copy()
avg_basket["avg_basket"] = avg_basket["monetary"] / frequency["frequency"]

FEATURES COMPORTEMENTALES

Nombre de catégories distinctes achetées

In [39]:
distinct_categories = (
    obs_data.groupby("customer_id")["product_code"]
    .nunique()
    .reset_index(name="distinct_categories")
)

Régularité des achats (écart-type des délais)

In [40]:
obs_data = obs_data.sort_values(["customer_id", "invoice_date"])

obs_data["prev_date"] = obs_data.groupby("customer_id")["invoice_date"].shift(1)
obs_data["days_between"] = (obs_data["invoice_date"] - obs_data["prev_date"]).dt.days

regularity = (
    obs_data.groupby("customer_id")["days_between"]
    .std()
    .reset_index(name="purchase_regularity")
)

regularity["purchase_regularity"] = regularity["purchase_regularity"].fillna(0)

Nombre de mois avec achat

In [41]:
obs_data["month"] = obs_data["invoice_date"].dt.to_period("M")

months_with_purchase = (
    obs_data.groupby("customer_id")["month"]
    .nunique()
    .reset_index(name="months_with_purchase")
)

**3.FEATURES TEMPORELLES**

**Ancienneté (tenure)**

In [42]:
tenure = (
    obs_data.groupby("customer_id")["invoice_date"]
    .min()
    .reset_index()
)

tenure["tenure_days"] = (snapshot_date - tenure["invoice_date"]).dt.days
tenure = tenure[["customer_id", "tenure_days"]]

**Mois du premier achat**

In [43]:
first_purchase_month = (
    obs_data.groupby("customer_id")["invoice_date"]
    .min()
    .reset_index()
)

first_purchase_month["first_month"] = first_purchase_month["invoice_date"].dt.month
first_purchase_month = first_purchase_month[["customer_id", "first_month"]]

**4.FEATURES SAISONNALITÉ**

**Achats en mois de pic**

In [44]:
obs_data["month"] = obs_data["invoice_date"].dt.month

peak_month = 12

seasonality = (
    obs_data.groupby("customer_id")
    .apply(lambda x: (x["month"] == peak_month).mean())
    .reset_index(name="peak_month_ratio")
)

/tmp/ipython-input-175/2099936538.py:7: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: (x["month"] == peak_month).mean())


**FEATURE GÉOGRAPHIQUE**

In [45]:
if "country" in obs_data.columns:
    country_dummies = pd.get_dummies(obs_data[["customer_id", "country"]].drop_duplicates(),
                                     columns=["country"])

    country_features = country_dummies.groupby("customer_id").max().reset_index()

**6.FUSION DES FEATURES (dataset final)**

In [46]:
features = recency.merge(frequency, on="customer_id")
features = features.merge(monetary, on="customer_id")
features = features.merge(avg_basket, on="customer_id")
features = features.merge(distinct_categories, on="customer_id")
features = features.merge(regularity, on="customer_id")
features = features.merge(months_with_purchase, on="customer_id")
features = features.merge(tenure, on="customer_id")
features = features.merge(first_purchase_month, on="customer_id")
features = features.merge(seasonality, on="customer_id")

# si country
if 'country_features' in locals():
    features = features.merge(country_features, on="customer_id", how="left")

**7. FUSION AVEC LA TARGET**

In [47]:
dataset_ml = features.merge(clv_future, on="customer_id")

In [49]:
dataset_ml = dataset_ml.rename(columns={"monetary_x": "monetary"})
dataset_ml.drop(columns=["monetary_y"], inplace=True)

**Visualisation du résultat Final**

In [50]:
print(dataset_ml.head())

   customer_id  recency_days  frequency  monetary  avg_basket  \
0      23259.0            44          1      5.95        5.95   
1      23311.0           198          1      2.50        2.50   
2      23455.0            19          1      3.78        3.78   
3      23525.0            31          1      9.90        9.90   
4      23572.0            42          1     13.92       13.92   

   distinct_categories  purchase_regularity  months_with_purchase  \
0                    1                  0.0                     1   
1                    1                  0.0                     1   
2                    1                  0.0                     1   
3                    1                  0.0                     1   
4                    1                  0.0                     1   

   tenure_days  first_month  ...  country_Cyprus  country_Denmark  \
0           44            5  ...           False            False   
1          198           12  ...           False        